# Generator kart punktów na Bieg Nocny
Żeby uruchomić generator musisz kliknąć symbol startu obok komórek z programem. Wykonaj je **PO KOLEI**.

In [1]:
#@title Inicjalizacja generatora
!pip install python-docx -q
import os
import sys
if not os.path.exists('BiegNocny_Tools'):
  !git clone https://github.com/cichy30002/BiegNocny_Tools.git -q
sys.path.append('/content/BiegNocny_Tools/Trasy')
import pandas as pd
import numpy as np
from google.colab import files
import docx
from Cyphers import CezarCypher, CellphoneCypher, FractionCypher, ReverseWordsCypher, MoorseCypher, SyllabeCypher, Cypher
import copy
from docx.enum.text import WD_BREAK
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#@title Wczytaj pliki wejścia
#@markdown 1. Wzór karty punktu w formacie .docx
#@markdown 2. Plik .xlsx zawierający arkusz stylu i arkusz trasy. Ustaw nazwy tych arkuszy.
uploaded = files.upload()

if not uploaded:
    raise ValueError("Please upload a file before continuing.")

template_filename = list(uploaded.keys())[0]

if not template_filename.lower().endswith('.docx'):
    raise ValueError("The uploaded file is not a .docx file. Please upload a .docx file.")

try:
    template = docx.Document(template_filename)
except Exception as e:
    raise ValueError("Could not read .docx file. Please check your file format.") from e

table = template.tables[0]

route_sheet_name = "Trasa R" #@param {"type":"string"}
style_sheet_name = "Styl" #@param {"type":"string"}

uploaded = files.upload()

if not uploaded:
    raise ValueError("Please upload a file before continuing.")

source_filename = list(uploaded.keys())[0]

if not source_filename.lower().endswith('.xlsx'):
    raise ValueError("The uploaded file is not a .xlsx file. Please upload a .xlsx file.")

try:
    style_df = pd.read_excel(source_filename, sheet_name=style_sheet_name, engine='openpyxl')
    route_df = pd.read_excel(source_filename, sheet_name=route_sheet_name, engine='openpyxl')
except Exception as e:
    raise ValueError("Could not read .docx file. Please check your file format.") from e

df = route_df.merge(style_df, on="ID Pytania", how="left")
df.head()

Saving template.docx to template.docx


Saving Punkty.xlsx to Punkty.xlsx


,NR Punktu,Kod,Nazwa Punktu,Współrzędna N,Współrzędna E,ID Pytania,Odpowiedź,Miejsce,Opis,Pytanie,Poprawna odpowiedź,Błędna odpowiedź 1,Błędna odpowiedź 2,Błędna odpowiedź 3
0,1,AYQ,Hufiec ZHP Poznań-Rejon,52.394037,16.903264,P1,B,Tablica wildeckich harcerzy poległych w latach...,Odsłonięta była w 1976 r. na ścianie siedziby ...,Ile nazwisk zawiera aktualnie zawiera Tablica ...,34,32,31,33
1,2,AOS,Emilii Sczanieckiej,52.395139,16.895029,P2,D,Tablica pamięci hm. Tadeusza Zielińskiego,Została odsłonięta w 1983 roku z okazji nadani...,W którym roku została odsłonięta Tablica Pamię...,1983,1981,1980,1984
2,3,ATD,Szymborska,52.392371,16.888367,P3,D,Pomnik pamięci harcerzy 7 HH,Pomnik znajduje się obok harcówki Hufca Poznań...,Co w obecnym pomniku Pamięci Harcerzy 7 HH jes...,czarna płyta z listą nazwisk,Obelisk,"marmurowy prostokąt, na którym stoi obelisk",płyta znajdująca się na przodzie obeliska
3,4,ARL,Stadion Grunwaldu,52.396670,16.883460,P4,D,Pomnik pamięci dh Henryka Wysockiego i Edwarda...,Henryk Wysocki HO urodził się 23 maja 1922 rok...,Co robili druhowie Henryk Wysocki HO i Edward ...,pełnili służbę obserwacyjno- meldunkową w Poz...,działali w Pogotowiu Harcerzy,pojechali na obronę Warszawy,brali udział w tajnym zebraniu harcerskim
4,5,AUB,Kościół św. Jana Kantego,52.398094,16.875550,P5,C,Tablica upamiętniająca miejsce pierwszej harcó...,Z okazji 80-tej rocznicy powstania pierwszego...,Z jakiej okazji została ufundowana tablica upa...,80-tej rocznicy powstania pierwszego zastępu,wybrania nowego komendanta hufca,Powstania nowej drużyny w hufcu,Przeniesienia miejsca harcówki


In [2]:
#@title Generowanie kart punktów
#@markdown Ustaw nazwę trasy i uruchom komórkę żeby wygenerować karty punktów.

cyphers = np.array([CezarCypher(3),
                    CellphoneCypher(),
                    FractionCypher(),
                    ReverseWordsCypher(),
                    MoorseCypher(),
                    SyllabeCypher("GADERYPOLUKI"),
                    SyllabeCypher("NOWEBUTYLISA"),
                    SyllabeCypher("POLITYKARENU"),
                    SyllabeCypher("KONIECMATURY"),
                    SyllabeCypher("KACEMINUTOWY"),
                    SyllabeCypher("KALINOWEBUTY")
])

def random_draw_no_repeat(arr, n):
    result = np.empty(n, dtype=arr.dtype)
    result[0] = np.random.choice(arr)
    for i in range(1, n):
        # choose from all except the previous value
        choices = arr[arr != result[i-1]]
        result[i] = np.random.choice(choices)
    return result

def encrypt_point(point_name: str, cell: docx.table._Cell, cyphers: list[Cypher]) -> None:
    cypher = cyphers.pop(0)
    if isinstance(cypher, FractionCypher):
        cell.text = ""
        oMath = cypher.encrypt_oMath(point_name)
        p = cell.add_paragraph()
        r = p.add_run()

        r._r.append(oMath)
        r.add_break(WD_BREAK.LINE)
    else:
        encrypted_text = cypher.encrypt(point_name)
        cell.text = encrypted_text

def format_question(point: pd.core.series.Series, previous_point: pd.core.series.Series) -> str:
    if pd.isna(previous_point["Pytanie"]):
        return "no question"

    question = previous_point["Pytanie"] + "\n"
    if previous_point["Odpowiedź"] not in ['A', 'B', 'C', 'D']:
        return question

    wrong_answers = [previous_point["Błędna odpowiedź 1"], previous_point["Błędna odpowiedź 2"], previous_point["Błędna odpowiedź 3"]]
    answer_letters = ['A', 'B', 'C', 'D']
    for letter in answer_letters:
        if point["Odpowiedź"] == letter:
            question += f"{letter}) {previous_point['Poprawna odpowiedź']}\n"
        else:
            wrong_answer = wrong_answers.pop(0)
            question += f"{letter}) {wrong_answer}\n"
    return question.strip()

def append_noncollapsible_paragraph(out_doc, page_break=False):
    # Create a paragraph XML node containing a non-breaking space so Word won't collapse it
    p = OxmlElement('w:p')
    r = OxmlElement('w:r')
    t = OxmlElement('w:t')
    t.text = '\u00A0'           # non-breaking space
    r.append(t)
    p.append(r)

    if page_break:
        # Add a run with a page break
        r_br = OxmlElement('w:r')
        br = OxmlElement('w:br')
        br.set(qn('w:type'), 'page')
        r_br.append(br)
        p.append(r_br)

    out_doc._body._body.append(p)

def set_text_with_formatting(cell: docx.table._Cell, text: str, bold=False, underline=False, centered=False) -> None:
    p = cell.paragraphs[0] if cell.paragraphs else cell.add_paragraph()
    p.text = str(text)
    p.runs[0].bold = bold
    p.runs[0].underline = underline
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER if centered else WD_ALIGN_PARAGRAPH.LEFT


route_name = "Trasa rowerowa 2025" #@param {"type":"string"}

n_points = df.shape[0]
cypher_sequence = random_draw_no_repeat(cyphers, n_points).tolist()
output_doc = docx.Document()

for i, point in df.iterrows():

    set_text_with_formatting(table.cell(1,0), "Punkt " + str(point["NR Punktu"]), bold=True, centered=True)
    set_text_with_formatting(table.cell(1,1), point["Kod"])
    set_text_with_formatting(table.cell(0,2), point["Nazwa Punktu"], bold=True, underline=True, centered=True)
    set_text_with_formatting(table.cell(1,2), point["Miejsce"], bold=True, centered=True)
    set_text_with_formatting(table.cell(1,3), route_name, bold=True, centered=True)
    set_text_with_formatting(table.cell(2,4), point["Opis"])
    set_text_with_formatting(table.cell(3,4), format_question(point, df.iloc[i-1] if i > 0 else df.iloc[-1]))
    nextpoint = df.iloc[i+1] if i+1 < n_points else df.iloc[0]
    encrypt_point(nextpoint["Nazwa Punktu"], table.cell(4,4), cypher_sequence)

    tbl = copy.deepcopy(table._tbl)
    output_doc._body._body.append(tbl)
    append_noncollapsible_paragraph(output_doc, page_break=(i%2 == 1 and i != n_points - 1))

NameError: name 'CezarCypher' is not defined

In [6]:
#@title Zapisz i pobierz
output_doc_filename = "Karty punktow Trasa Rowerowa 2025.docx" #@param {type:"string"}
output_doc.save(output_doc_filename)
files.download(output_doc_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>